In [4]:
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import plotly.offline as plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
yf.pdr_override()
import os
plotly.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'
import wrds
import numpy as np

In [5]:
df_adj_close_clean = pd.read_csv('data/df_adj_close_clean.csv')
df_open_clean = pd.read_csv('data/df_open_clean.csv')
df_volume_clean = pd.read_csv('data/df_volume_clean.csv')
df_shares_outstanding_clean = pd.read_csv('data/df_shares_outstanding_clean.csv')
df_volume_clean_wrds = pd.read_csv('data/df_volume_clean_WRDS.csv')

In [6]:
# change index to datetime
df_adj_close_clean.index = pd.to_datetime(df_adj_close_clean['Date'])
df_open_clean.index = pd.to_datetime(df_open_clean['Date'])
df_volume_clean.index = pd.to_datetime(df_volume_clean['Date'])
df_shares_outstanding_clean.index = pd.to_datetime(df_shares_outstanding_clean['Date'])
df_volume_clean_wrds.index = pd.to_datetime(df_volume_clean_wrds['Date'])


In [7]:
df_adj_close_clean = df_adj_close_clean.drop(columns=['Date'])
df_open_clean = df_open_clean.drop(columns=['Date'])
df_volume_clean = df_volume_clean.drop(columns=['Date'])
df_shares_outstanding_clean = df_shares_outstanding_clean.drop(columns=['Date'])
df_volume_clean_wrds = df_volume_clean_wrds.drop(columns=['Date'])

# Velocity calculation

We do the velocity calculation in a fashion similar as Feng et. al's 2012 paper, in the following steps:
1. Divide the total number of shares traded on the market(volume) by the number of outstanding shares of each stock every day
2. Resample the dataframe into a yearly timeframe
3. Sum up the yearly velocity per stock and divide it by the amount of years used in the dataframe which results in the mean velocity per stock
4. Take the mean of all the mean of the different stocks

In [35]:
# Step 1
velocity_df_daily = df_volume_clean_wrds/df_shares_outstanding_clean

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-12-31,2.243222,47.472133,2.696070,3.932879,0.995654,2.170546,1.299114,0.835836,1.212257,1.808208,...,2.102904,3.120338,1.680767,0.716516,1.357763,1.707823,2.040663,1.565663,0.864328,2.841228
2014-12-31,1.718739,6.183702,2.746696,2.611206,1.445748,1.890528,0.995593,0.974616,1.115510,1.697390,...,2.062747,4.467449,1.374725,0.688893,1.480078,1.390884,1.801760,1.845859,2.347544,3.229422
2015-12-31,1.933402,4.250476,3.647942,2.267491,1.634276,2.371917,0.980258,0.915805,0.998797,1.410080,...,1.817627,7.964470,1.642079,0.846755,1.704901,1.582251,2.374838,1.666949,2.585738,3.351885
2016-12-31,1.606480,4.061178,3.542195,1.766577,1.232397,2.928942,1.548841,0.781507,1.030535,1.425723,...,1.794005,8.466408,1.621252,0.752480,1.994415,1.847233,2.196985,1.978114,3.123899,4.144796
2017-12-31,1.440190,2.971288,5.635287,1.303305,0.936932,2.052232,0.960550,0.902079,0.856854,1.249402,...,1.086882,4.879674,1.299619,0.650838,1.770794,1.647258,1.535732,1.692205,1.559585,3.078645
2018-12-31,1.991302,3.686198,3.883152,1.725799,1.128780,1.780829,0.886048,0.971949,0.882577,1.668605,...,1.491753,7.414999,1.812973,0.799005,3.017358,1.540262,1.636458,1.473710,1.888861,3.020075
2019-12-31,1.722800,3.931448,3.964397,1.532499,1.328858,1.594778,0.735507,0.773981,0.732547,1.373500,...,1.394492,4.751873,1.563891,0.690505,1.862578,1.178269,1.408720,1.394899,2.609203,3.117888
2020-12-31,1.513987,35.930756,3.607567,2.293588,1.467260,1.591052,0.956960,1.269450,0.920433,1.610509,...,1.710313,11.146641,1.479746,1.720323,2.075322,1.642990,1.805867,1.684287,1.928686,3.933323
2021-12-31,1.343329,15.085496,2.981251,1.364593,0.970714,1.195868,0.759534,1.285585,0.755593,1.248716,...,1.461972,7.341155,1.380324,1.421535,1.729708,1.212474,1.302570,1.659944,1.319745,2.069073


In [37]:
#Step 2
velocity_yearly_df = velocity_df.resample('Y').sum()
velocity_yearly_df

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-12-31,2.243222,47.472133,2.696070,3.932879,0.995654,2.170546,1.299114,0.835836,1.212257,1.808208,...,2.102904,3.120338,1.680767,0.716516,1.357763,1.707823,2.040663,1.565663,0.864328,2.841228
2014-12-31,1.718739,6.183702,2.746696,2.611206,1.445748,1.890528,0.995593,0.974616,1.115510,1.697390,...,2.062747,4.467449,1.374725,0.688893,1.480078,1.390884,1.801760,1.845859,2.347544,3.229422
2015-12-31,1.933402,4.250476,3.647942,2.267491,1.634276,2.371917,0.980258,0.915805,0.998797,1.410080,...,1.817627,7.964470,1.642079,0.846755,1.704901,1.582251,2.374838,1.666949,2.585738,3.351885
2016-12-31,1.606480,4.061178,3.542195,1.766577,1.232397,2.928942,1.548841,0.781507,1.030535,1.425723,...,1.794005,8.466408,1.621252,0.752480,1.994415,1.847233,2.196985,1.978114,3.123899,4.144796
2017-12-31,1.440190,2.971288,5.635287,1.303305,0.936932,2.052232,0.960550,0.902079,0.856854,1.249402,...,1.086882,4.879674,1.299619,0.650838,1.770794,1.647258,1.535732,1.692205,1.559585,3.078645
2018-12-31,1.991302,3.686198,3.883152,1.725799,1.128780,1.780829,0.886048,0.971949,0.882577,1.668605,...,1.491753,7.414999,1.812973,0.799005,3.017358,1.540262,1.636458,1.473710,1.888861,3.020075
2019-12-31,1.722800,3.931448,3.964397,1.532499,1.328858,1.594778,0.735507,0.773981,0.732547,1.373500,...,1.394492,4.751873,1.563891,0.690505,1.862578,1.178269,1.408720,1.394899,2.609203,3.117888
2020-12-31,1.513987,35.930756,3.607567,2.293588,1.467260,1.591052,0.956960,1.269450,0.920433,1.610509,...,1.710313,11.146641,1.479746,1.720323,2.075322,1.642990,1.805867,1.684287,1.928686,3.933323
2021-12-31,1.343329,15.085496,2.981251,1.364593,0.970714,1.195868,0.759534,1.285585,0.755593,1.248716,...,1.461972,7.341155,1.380324,1.421535,1.729708,1.212474,1.302570,1.659944,1.319745,2.069073


In [52]:
# Step 3 + 4
sum_velocity_yearly_df = velocity_yearly_df.apply(lambda x: x.sum())/velocity_yearly_df.shape[0] # .shape[0] = amount of rows 
sum_velocity_yearly_df.mean()

2.183549851113631